In [1]:
%matplotlib inline

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from cv2 import VideoWriter, VideoWriter_fourcc
import pandas as pd
import torch
from scipy import signal, ndimage, spatial
from scipy.signal import correlate
from scipy.interpolate import CubicSpline
from scipy.ndimage import gaussian_filter
import math 
from signal_alignment import phase_align, chisqr_align
from scipy.interpolate import interp1d
import scipy.stats as stats
from scipy import fftpack
from scipy.stats import spearmanr

### Function Definitions 

In [3]:
def cleaning_wave_df(DF):
    # dropping columns with no value
    #DF = DF.dropna(axis =1) 
    # dropping wave id column
    DF = DF.drop(columns = 2, axis =1)
    # name the first sensor columns 
    cols_0={0: "Time", 1: "FrameID", 3: "SensorID", 4: "Sensor_1Status", 5: "X_Nose", 6: "Y_Nose", 7:"Z_Nose"}
    DF = DF.rename(columns=cols_0, errors="raise")
    DF = DF.drop(columns = [8,9,10, 11], axis =1)
    
    DF = DF.drop(columns = [12,13,14,15,16,17,18,19,20], axis =1)
    cols_1={21: "SensorID", 22: "Sensor_2Status", 23: "X_LLeft", 24: "Y_LLeft", 25:"Z_LLeft"}
    DF = DF.rename(columns=cols_1, errors="raise")
    
    DF = DF.drop(columns = [26,27,28,29], axis =1)
    cols_2={30: "SensorID", 31: "Sensor_3Status", 32: "X_LR", 33: "Y_LR", 34:"Z_LR"}
    DF = DF.rename(columns=cols_2, errors="raise")
    
    DF = DF.drop(columns = [35,36,37,38], axis =1)
    cols_3={39: "SensorID", 40: "Sensor_4Status", 41: "X_UL", 42: "Y_UL", 43:"Z_UL"}
    DF = DF.rename(columns=cols_3, errors="raise")
    
    DF = DF.drop(columns = [44,45,46,47], axis =1)
    cols_4={48: "SensorID", 49: "Sensor_5Status", 50: "X_LL", 51: "Y_LL", 52:"Z_LL"}
    DF = DF.rename(columns=cols_4, errors="raise")
    
    DF = DF.drop(columns = [53,54,55,56], axis =1)
    cols_5={57: "SensorID", 58: "Sensor_6Status", 59: "X_JR", 60: "Y_JR", 61:"Z_JR"}
    DF = DF.rename(columns=cols_5, errors="raise")
    
    DF = DF.drop(columns = [62,63,64,65], axis =1)
    cols_6={66: "SensorID", 67: "Senson_7Status", 68: "X_JL", 69: "Y_JL", 70:"Z_JL"}
    DF = DF.rename(columns=cols_6, errors="raise")
    DF = DF.drop(columns = [71,72,73,74], axis =1)
    return DF

# cleaning video data 

def cleaning_video_df(DF):
    
    # choose the needed columns and convert the values to mm
    DF = DF[["BAG_Frame_number",'Time_Stamp (s)','Video_Frame_number',
             'landmark_48', 'landmark_48.1', 'landmark_48.2', 'landmark_54',
            'landmark_54.1', 'landmark_54.2', 'landmark_51', 'landmark_51.1',
            'landmark_51.2', 'landmark_57', 'landmark_57.1', 'landmark_57.2']]
  
    
    DF = DF.rename(columns={'Time_Stamp (s)': 'Time', "landmark_48": "X_LR", "landmark_48.1": "Y_LR",
                           "landmark_48.2": "Z_LR", "landmark_54": "X_LLeft",
                           "landmark_54.1": "Y_LLeft", "landmark_54.2": "Z_LLeft",
                           "landmark_51": "X_UL", "landmark_51.1": "Y_UL",
                           "landmark_51.2": "Z_UL", "landmark_57": "X_LL", "landmark_57.1": "Y_LL",
                           "landmark_57.2": "Z_LL"})
    
    DF = DF.astype({ "X_LR": np.double, "Y_LR": np.double,"Z_LR": np.double, "X_LLeft": np.double,
                            "Y_LLeft": np.double, "Z_LLeft": np.double, "X_UL": np.double,  "Y_UL": np.double,
                            "Z_UL": np.double,  "X_LL": np.double,  "Y_LL": np.double, "Z_LL": np.double})
    
    # conver the values from m to mm
    DF[["X_LR", "Y_LR","Z_LR", "X_LLeft", "Y_LLeft", "Z_LLeft", "X_UL",  "Y_UL","Z_UL",  "X_LL", 
       "Y_LL", "Z_LL"]] =  DF[["X_LR", "Y_LR","Z_LR", "X_LLeft", "Y_LLeft", "Z_LLeft", "X_UL",  "Y_UL","Z_UL",  "X_LL", 
       "Y_LL", "Z_LL"]]*1000
    return DF

def area_of_triangle(A,B,C):

    """
    computes the area of a triangle given by 3 points in 2d or 3d

    A, B and C must be numpy arrays

    it works with vectors A=[a1,a2,a3], B=[b1,b2,b3], c=[c1,c2,c3] or matrices

    A=[[a11,a12,a13],...[a1n,a2n,b3n]], B=[[b11,b12,b13],...[b1n,b2n,b3n]], C=[[c11,c12,c13],...[c1n,c2n,c3n]]

    """

    As = A.shape
    Bs = B.shape
    Cs = C.shape
   
    if len(As) == 1 :

        #we got vectors

        if (As[0]>3) or (Bs[0]>3) or (Cs[0]>3):

            raise Exception('coordinates can only be 2d or 3d')

            return None

    else:

        #check at least one of the dimensions is two or three
        if (As[0]==2) or (As[0]==3) or (As[1]==2) or (As[1]==3):

            #one of the dimensions of A is 2 or 3, now check that all the vectors have the same size
            if (As!=Bs) or (As!=Cs):

                raise Exception('vectors must be the same size')
                return None

            else:
                #move forward
                pass

        else:

            raise Exception('coordinates can only be 2d or 3d')

            return None

    #at this point we know that one of the dimensions has 2 or 3 elements we move forward assuming that
    #the user provided the vectors with the correct size
    #move all vectors to the same origin

    AB = B-A
    AC = C-A  

    if len(As) == 1 :

        #if only one vector the simply compute the norm of the cross product
        area = (1/2)*np.linalg.norm(np.cross(AB,AC))

    else:

        #if only multiple vectors compute the norm along the axis one
        area = (1/2)*np.linalg.norm(np.cross(AB,AC), axis = 1)   

    return area

def lipDist(DF):
    
    # Assumes columns are named in a particular way 
    
    DF['Horiz_Lip_Motion'] = DF.apply(lambda row: math.sqrt((row.X_LLeft - row.X_LR)**2 + (row.Y_LLeft - row.Y_LR)**2
                                                                  + (row.Z_LLeft - row.Z_LR)**2), axis = 1) 
    DF['Vert_Lip_Motion'] = DF.apply(lambda row: math.sqrt((row.X_UL - row.X_LL)**2 + (row.Y_UL - row.Y_LL)**2 +
                                                                 (row.Z_UL - row.Z_LL)**2), axis = 1)
    return DF

In [4]:
def findPeakIndex(array, minThreshold = 50, widthapart = 50,  twosided = True):
    
    peaks, _ = signal.find_peaks(array, height= minThreshold, distance= widthapart)
    
    if twosided == True:
        peaks_neg, _ = signal.find_peaks(-array, height= minThreshold, distance= widthapart)
        peaks = np.concatenate((peaks, peaks_neg), axis=None)
        peaks = np.sort(peaks, axis=None)
    
    return peaks
        

def rom(list_of_arrays):

    
    reps_max_values = np.array([np.max(array) for array in list_of_arrays])
    reps_min_values = np.array([np.min(array) for array in list_of_arrays])
    
    
    ROM = reps_max_values - reps_min_values
    
    return ROM
    

In [5]:
# Signal Processing

# sampling frequency set to 100 HZ
fsample = 100

def sig_shift(signal, amount):
    length = len(signal)    
    shifted = np.zeros(length+amount - 1)
    shifted[:amount-1] = 0
    shifted[amount-1:] = signal[:]
    return shifted

def sig_norm(signal):
    signal_mean = np.mean(signal)
    signal_normalised = signal - signal_mean
    return signal_normalised

# Butterworth filter 8-order, 15hz cutoff
fc = 15
w = fc / (fsample/2)
b , a = signal.butter(8, w, btype ='lowpass')

### Reading, Cleaning, and Preprocessing DATA

In [6]:
Tasks = ['REST', 'OPEN', 'SPREAD', 'OOEE', 'PA', 'BBP', 'TNG_PROTRUSION', 'TNG_LAT', 'TNG_NOSE']
Conditions = ['FAST', 'SLOW', 'DIS', 'HOLD', 'NORM']

TASK_DFS_VIDEO_LIST = list()
TASK_DFS_WAVE_LIST = list()

In [7]:
# choose a task to do the analysis on,for example open, spread, pa, bbp, etc
Task = Tasks[1]

In [8]:
#####   WAVE DATA   ######

# read the wave data files of the chosen task into dfs and store them all in a list 
path_w = r'C:\Users\jafarid\Documents\Code\ValidationStudy\wave_data'
ext_w=('.tsv')
Files_w = os.listdir(path_w)           
Files_w = [i for i in Files_w if i.endswith(ext_w)]

for i in range(0, len(Files_w)):
    # choose which task to focus on 
    if Task in Files_w[i]:
        df = pd.read_csv(path_w+ "\\" + Files_w[i], delimiter='\t', skiprows=1,header=None)
        df = cleaning_wave_df(df)
        # depending on the file we are reading the indexing below needs to change 
        #if Files_w[i][20:-4] == 'EN_HOLD_1' or Files_w[i][20:-4] == 'EN_HOLD':
        #    df['FileName'] = Files_w[i][18:-4]
        
        #else: 
        df['FileName'] = Files_w[i][18:-4]
        df['PatientID'] = Files_w[i][:2]
        df['DataDATE'] = Files_w[i][5:13]
        df['Task'] = Task
        for condition in Conditions:
            if condition in Files_w[i]:
                df['Condition'] = condition

            else:
                df['Condition'] = 'NORM'
                
   
        # preprocessing, using the cubic interpolation to fill in the missing data 
        df = df.interpolate(method ='cubic', limit_direction ='forward') 
        df = lipDist(df)

        #left_area = area_of_triangle(df[["X_LLeft", "Y_LLeft", "Z_LLeft"]].values, df[["X_UL",  "Y_UL","Z_UL"]].values, 
        #                               df[["X_LL", "Y_LL", "Z_LL"]].values)
        #df['Area_Left'] = np.array(left_area)
        #right_area = area_of_triangle(df[["X_LR", "Y_LR","Z_LR"]].values, df[["X_UL",  "Y_UL","Z_UL"]].values, 
        #                                df[[ "X_LL", "Y_LL", "Z_LL"]].values)
        #df['Area_Right'] =  np.array(right_area)        
        #df["Area_Ratio"] = df.apply(lambda row: row.Area_Left/row.Area_Right, axis = 1)
        
        # Gaussian filter
        
        df['Horiz_Lip_Motion_F1'] = signal.filtfilt(b, a, df['Horiz_Lip_Motion'])
        
        df["Speed"] = np.gradient(df['Vert_Lip_Motion'], df["Time"])
        
        # Butterworth filter 8-order, 15hz cutoff
        df['Vert_Lip_Motion_F1']= signal.filtfilt(b, a, df['Vert_Lip_Motion'])
        #df['Vert_Lip_Motion_F2']= gaussian_filter(df['Vert_Lip_Motion_F1'], sigma=3)
        # speed
        #df["Speed_F1"] = signal.filtfilt(b, a, np.gradient(df['Vert_Lip_Motion'], df["Time"]))
        
        TASK_DFS_WAVE_LIST.append(df)
        
        print(Files_w[i][:2],Files_w[i][18:-4])


    


DJ OPEN_HOLD
DJ OPEN_NORM
DJ OPEN_DIS
DJ OPEN_FAST
MK OPEN_HOLD
MK OPEN_NORM
MK OPEN_DIS
MK OPEN_FAST
RM OPEN_HOLD
RM OPEN_NORM
RM OPEN_DIS
RM OPEN_FAST


In [9]:
if 0:
    %matplotlib qt
    plt.plot(TASK_DFS_WAVE_LIST[3]['Vert_Lip_Motion'],'b')
    plt.plot(TASK_DFS_WAVE_LIST[3]['Vert_Lip_Motion_F1'],'g')
    plt.plot(TASK_DFS_WAVE_LIST[3]['Vert_Lip_Motion_F2'],'r')

In [10]:
#####   VIDEO DATA   ######

# read the wave data files of the chosen task into dfs and store them all in a list 
path_v = r'C:\Users\jafarid\Documents\Code\ValidationStudy\video_data_processed'
ext_v=('_Landmarks3D.csv')
Files_v = os.listdir(path_v)           
Files_v = [i for i in Files_v if i.endswith(ext_v)]

for i in range(0, len(Files_v)):
    # choose which task to focus on 
    if Task in Files_v[i]:
        df = pd.read_csv(path_v+ "\\" + Files_v[i])
        df =  df.drop([0]) 
        df = cleaning_video_df(df)
        # depending on the file we are reading the indexing below needs to change 
        df['FileName'] = Files_v[i][12:-16]
        df['PatientID'] = Files_v[i][:2]
        #df['DataDATE'] = Files_v[i][5:13]
        df['Task'] = Task
        for condition in Conditions:
            if condition in Files_v[i]:
                df['Condition'] = condition

            else:
                df['Condition'] = 'NORM'
                

        # preprocessing, using the cubic interpolation to fill in the missing data 
        df = df.interpolate(method ='cubic', limit_direction ='forward') 
        df = lipDist(df)

        #left_area = area_of_triangle(df[["X_LLeft", "Y_LLeft", "Z_LLeft"]].values, df[["X_UL",  "Y_UL","Z_UL"]].values, 
        #                               df[["X_LL", "Y_LL", "Z_LL"]].values)
        #df['Area_Left'] = np.array(left_area)
        #right_area = area_of_triangle(df[["X_LR", "Y_LR","Z_LR"]].values, df[["X_UL",  "Y_UL","Z_UL"]].values, 
        #                                df[[ "X_LL", "Y_LL", "Z_LL"]].values)
        #df['Area_Right'] =  np.array(right_area)        
        #df["Area_Ratio"] = df.apply(lambda row: row.Area_Left/row.Area_Right, axis = 1)
        # Butterworth filter 8-order, 15hz cutoff
        
        df['Vert_Lip_Motion_F1']= signal.filtfilt(b, a, df['Vert_Lip_Motion'])  
        df['Horiz_Lip_Motion_F1'] = signal.filtfilt(b, a, df['Horiz_Lip_Motion'])
        df["Speed"] = np.gradient(df['Vert_Lip_Motion_F1'], df["Time"])
        
       # df['Vert_Lip_Motion_F3']= gaussian_filter(df['Vert_Lip_Motion_F1'], sigma=3)
       # df['Vert_Lip_Motion_F2']= signal.medfilt(df['Vert_Lip_Motion_F1'], 3)
        
        #df['Horiz_Lip_Motion_F1']= gaussian_filter(df['Horiz_Lip_Motion'], sigma=3)
        
        TASK_DFS_VIDEO_LIST.append(df)
        print(Files_v[i][12:-16])

    


OPEN_DIS
OPEN_FAST
OPEN_HOLD
OPEN_NORM
OPEN_DIS
OPEN_FAST
OPEN_HOLD
OPEN_NORM
OPEN_DIS
OPEN_FAST
OPEN_HOLD
OPEN_NORM


In [11]:
if 0:
    %matplotlib qt
    plt.plot(TASK_DFS_VIDEO_LIST[2]['Vert_Lip_Motion'],'o')
    plt.plot(TASK_DFS_VIDEO_LIST[2]['Vert_Lip_Motion_F1'],'g')
    plt.plot(TASK_DFS_VIDEO_LIST[2]['Vert_Lip_Motion_F2'],'r')
    plt.plot(TASK_DFS_VIDEO_LIST[2]['Vert_Lip_Motion_F3'],'b')

In [12]:

TASK_DFS_WAVE_100HZ_LIST = list()


In [13]:
# RESAMPLING TO 100Hz for the wave data

for DF_WAVE in TASK_DFS_WAVE_LIST:
    WAVE_100HZ = pd.DataFrame()
    tmax = len(DF_WAVE.Time) - 1
    tmax_value = DF_WAVE.Time[tmax]
    tmin_value = DF_WAVE.Time[0]
    nsample = fsample*(tmax_value - tmin_value)
    perfect_time = np.linspace(tmin_value, tmax_value, num= nsample, endpoint=True)
    WAVE_100HZ['Time'] = perfect_time
    f_motion = interp1d(DF_WAVE.Time, DF_WAVE['Vert_Lip_Motion_F1'], kind = 'linear')
    WAVE_100HZ['Vert_Lip_Motion'] = f_motion(perfect_time)
    #df['Horiz_Lip_Motion_F1']
    f_speed = interp1d(DF_WAVE.Time, DF_WAVE['Speed'], kind = 'linear')
    WAVE_100HZ['Speed'] = f_speed(perfect_time)
    WAVE_100HZ['FileName'] = DF_WAVE['FileName'][1]
    WAVE_100HZ['PatientID'] = DF_WAVE['PatientID'][1]
    TASK_DFS_WAVE_100HZ_LIST.append(WAVE_100HZ)
    #print(len(DF_WAVE['Vert_Lip_Motion_F1']))
    #print(len(WAVE_100HZ['Vert_Lip_Motion']))
    #print()

C:\Users\jafarid\AppData\Local\Continuum\anaconda3\envs\opecvpytorch_env\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  if __name__ == '__main__':


In [14]:
TASK_DFS_VIDEO_100HZ_LIST = list()

In [15]:
# Upsampling to 100Hz for the video data
for DF_VIDEO in TASK_DFS_VIDEO_LIST:
    WAVE_100HZ = pd.DataFrame()
    tmax = len(DF_VIDEO.Time) - 1
    tmax_value = DF_VIDEO.Time[tmax]
    tmin_value = DF_VIDEO.Time[1]
    nsample = fsample*(tmax_value - tmin_value)
    perfect_time = np.linspace(tmin_value, tmax_value, num= nsample, endpoint=True)
    WAVE_100HZ['Time'] = perfect_time
    f_motion = interp1d(DF_VIDEO.Time, DF_VIDEO['Vert_Lip_Motion_F1'], kind = 'linear')
    WAVE_100HZ['Vert_Lip_Motion'] = f_motion(perfect_time)
    f_speed = interp1d(DF_VIDEO.Time, DF_VIDEO['Speed'], kind = 'linear')
    WAVE_100HZ['Speed'] = f_speed(perfect_time)
    WAVE_100HZ['FileName'] = DF_VIDEO['FileName'][1]
    WAVE_100HZ['PatientID'] = DF_VIDEO['PatientID'][1]
    TASK_DFS_VIDEO_100HZ_LIST.append(WAVE_100HZ)

C:\Users\jafarid\AppData\Local\Continuum\anaconda3\envs\opecvpytorch_env\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  


In [16]:
if 0:
    %matplotlib inline
    plt.plot(TASK_DFS_VIDEO_100HZ_LIST[9]['Speed'],'r')
    plt.plot(TASK_DFS_WAVE_100HZ_LIST[11]['Speed'],'b')

    plt.figure()

    a = sig_norm(TASK_DFS_VIDEO_100HZ_LIST[9]['Speed'].values)
    b = sig_norm(TASK_DFS_WAVE_100HZ_LIST[11]['Speed'].values)
    plt.plot(a,'r')
    plt.plot(b,'b')

In [17]:
oh_pc = list()
on_pc = list()
od_pc = list()
of_pc = list()
oh_rms_list = list()
on_rms_list = list()
od_rms_list = list()
of_rms_list = list()
Final_DFS_LIST = list()

In [18]:
len(TASK_DFS_WAVE_100HZ_LIST)

12

In [19]:
for i in range(len(TASK_DFS_WAVE_100HZ_LIST)):
    for j in range(len(TASK_DFS_VIDEO_100HZ_LIST)):
        
        if TASK_DFS_WAVE_100HZ_LIST[i]['PatientID'][1]== TASK_DFS_VIDEO_100HZ_LIST[j]['PatientID'][1]:
        
            if TASK_DFS_WAVE_100HZ_LIST[i]['FileName'][1]== TASK_DFS_VIDEO_100HZ_LIST[j]['FileName'][1]:
                result = pd.DataFrame() 
                print(TASK_DFS_WAVE_100HZ_LIST[i]['FileName'][1])
                print('Wave index:{}'.format(i))
                print('Video index:{}'.format(j))

                video_n = sig_norm(TASK_DFS_VIDEO_100HZ_LIST[j]['Vert_Lip_Motion'].values)
                #v_n = sig_norm(TASK_DFS_VIDEO_100HZ_LIST[j]['Vert_Lip_Motion'].values)
                wave_n = sig_norm(TASK_DFS_WAVE_100HZ_LIST[i]['Vert_Lip_Motion'].values)
           #     video_n = video_n[200:-50]
           #     wave_n = wave_n[200:-50]
                time = TASK_DFS_WAVE_100HZ_LIST[i]['Time'].values
                time_w = TASK_DFS_WAVE_100HZ_LIST[i]['Time'].values
                time_v = TASK_DFS_WAVE_100HZ_LIST[i]['Time'].values
           #     time = time[200:-50]
                
                if len(video_n)<=len(wave_n):
                    upper_bound = len(video_n) 
                else: 
                    upper_bound = len(wave_n) 
                    
                if TASK_DFS_WAVE_100HZ_LIST[i]['FileName'][1]== "OPEN_NORM" and TASK_DFS_WAVE_100HZ_LIST[i]['PatientID'][1]== 'RM':
                    s = phase_align(wave_n[500:upper_bound], video_n, [5,upper_bound])
                    sp = math.ceil(s) + 500
                else:    

                    s = phase_align(wave_n, video_n, [5,upper_bound])
                print('The phase shift is:{}'.format(s))

                sp = math.ceil(s)


                video_shifted = sig_shift(TASK_DFS_VIDEO_100HZ_LIST[j]['Vert_Lip_Motion'],sp)
                video_speed_shifted = sig_shift(TASK_DFS_VIDEO_100HZ_LIST[j]['Speed'],sp)
                video_n_s = sig_shift(video_n, sp)
                wave_shifted = TASK_DFS_WAVE_100HZ_LIST[i]['Vert_Lip_Motion'].values
                wave_speed_shifted = TASK_DFS_WAVE_100HZ_LIST[i]['Speed'].values
                
                upper_bound_shifted = upper_bound + sp -1
                
                if len(wave_shifted)< upper_bound_shifted:
                    upper_bound_shifted = len(wave_shifted)
                
                
               # print('Video:{}'.format(len(video_shifted)))
               # print('WAVE:{}'.format(len(wave_shifted)))
               # print('UPPER_BOUND_SHIFTED:{}'.format(upper_bound_shifted))
               # print('UPPER_BOUND:{}'.format(upper_bound))
                
                name = TASK_DFS_WAVE_100HZ_LIST[i]['PatientID'][1] + '_' + TASK_DFS_WAVE_100HZ_LIST[i]['FileName'][1]
                
                range_l = sp+100
                range_u = upper_bound_shifted-10
                
                result['Time_WAVE'] = time_w[range_l:range_u]
                result['Time_VIDEO'] = time_v[range_l:range_u]
                #result['Time'] = time[range_l:range_u]
                result['VerDisp_WAVE'] = wave_shifted[range_l:range_u]
                result['VerDisp_VIDEO'] = video_shifted[range_l:range_u]
                
              #  result['HorDisp_WAVE']
              #  result['HorDisp_VIDEO']
                
                result['Speed_WAVE'] = wave_speed_shifted[range_l:range_u]
                result['Speed_VIDEO'] = video_speed_shifted[range_l:range_u]
                
                result['FileName'] = TASK_DFS_WAVE_100HZ_LIST[i]['FileName'][1]
                result['PatientID'] = TASK_DFS_WAVE_100HZ_LIST[i]['PatientID'][1]
                cvs = 'final_paper_data/OPEN_'+ name +'.csv'
                result.to_csv(cvs)
                Final_DFS_LIST.append(result)
                plt.figure()
                
               # plt.plot(time[sp:upper_bound_shifted], wave_shifted[sp:upper_bound_shifted],'b', label='Ground truth (WAVE)')
               # plt.plot(time[sp:upper_bound_shifted],video_shifted[sp:upper_bound_shifted],'r', label='Video + FAN tracking')
                plt.plot(time[range_l:range_u], wave_shifted[range_l:range_u],'b', label='Ground truth (WAVE)')
                plt.plot(time[range_l:range_u],video_shifted[range_l:range_u],'r', label='Video + FAN tracking')
                plt.legend(loc="upper left")
                plt.xlabel('Time (s)')
                plt.ylabel('Vertical Range of Motion (mm)')
                
                plt.savefig('{}'.format(name))
               # plt.plot(TASK_DFS_WAVE_100HZ_LIST[i]['Vert_Lip_Motion'],'b')
               # plt.plot(TASK_DFS_VIDEO_100HZ_LIST[j]['Vert_Lip_Motion'],'r')
                #plt.legend()
                #plt.figure()
                #a = np.gradient(TASK_DFS_WAVE_100HZ_LIST[i]['Time'])
                #plt.plot(a, 'b')
                #b = np.gradient(TASK_DFS_VIDEO_100HZ_LIST[j]['Time'])
                #plt.plot(b, 'r')
                


                plt.figure()
               # plt.plot(wave_n[100:upper_bound],'b')
               # plt.plot(video_n_s[100:upper_bound],'r')
                plt.plot(time[range_l:range_u],wave_n[range_l:range_u],'b', label='Normalized Ground truth (WAVE)')
                plt.plot(time[range_l:range_u],video_n_s[range_l:range_u],'r', label='Normalized Video + FAN tracking')
                plt.legend(loc="upper left")
                plt.xlabel('Time (s)')
                plt.ylabel('Vertical Range of Motion (mm)')
                plt.savefig('{}'.format(name+'Normalized'))
               # plt.figure()
               # plt.plot(wave_n[100:upper_bound],'b')
               # plt.plot(video_n_s[100:upper_bound],'r')
               # plt.plot(wave_n[sp:upper_bound_shifted],'b')
               # plt.plot(video_n_s[sp:upper_bound_shifted],'r')
                
                
              #  plt.plot(TASK_DFS_WAVE_100HZ_LIST[i]['Vert_Lip_Motion'],'b')
              #  plt.plot(video_shifted,'r')
                #plt.legend()
                
                plt.figure()
                plt.scatter(wave_shifted[range_l:range_u], video_shifted[range_l:range_u]) 
                
                
                r, p = stats.pearsonr(video_n_s[range_l:range_u], wave_n[range_l:range_u])
                rms = np.sqrt(np.mean((wave_n[range_l:range_u]-video_n_s[range_l:range_u])**2))
                #sum((abs(wave_n[range_l:range_u]-video_n_s[range_l:range_u])**2))/len(wave_n[range_l:range_u])
               # rms_list.append(rms)
                print('The corrolation r and p-value between the signals are: r{}, p{}'.format(r,p))
                print()
                if TASK_DFS_WAVE_100HZ_LIST[i]['FileName'][1] == "OPEN_HOLD":
                    oh_pc.append(r)
                    oh_rms_list.append(rms)
                    
                elif TASK_DFS_WAVE_100HZ_LIST[i]['FileName'][1] == "OPEN_DIS":
                    od_pc.append(r)
                    od_rms_list.append(rms)
                    
                elif TASK_DFS_WAVE_100HZ_LIST[i]['FileName'][1] == "OPEN_NORM":
                    on_pc.append(r)
                    on_rms_list.append(rms)
                else: 
                    of_pc.append(r)
                    of_rms_list.append(rms)



OPEN_HOLD
Wave index:0
Video index:2


KeyboardInterrupt: 

In [ ]:
print(oh_pc)
print(on_pc)
print(od_pc)
print(of_pc)

In [ ]:
import statistics 
from statistics import mean 
from statistics import stdev 
print(mean(oh_pc))
print(mean(on_pc))
print(mean(od_pc))
print(mean(of_pc))
print()
print(stdev(oh_pc))
print(stdev(on_pc))
print(stdev(od_pc))
print(stdev(of_pc))



In [ ]:
print(mean(of_rms_list),stdev(of_rms_list))
print(mean(oh_rms_list),stdev(oh_rms_list))
print(mean(od_rms_list),stdev(od_rms_list))
print(mean(on_rms_list),stdev(on_rms_list))

In [ ]:
for i in range(len(Final_DFS_LIST)):
    plt.figure()
    plt.plot(Final_DFS_LIST[i]['Time'], Final_DFS_LIST[i]['Speed_WAVE'],
             'b',label='Ground truth (WAVE)')
    plt.plot(Final_DFS_LIST[i]['Time'],Final_DFS_LIST[i]['Speed_VIDEO'],
             'r', label='Video + FAN tracking')
    plt.legend(loc="upper left")
    plt.xlabel('Time (s)')
    plt.ylabel('Vertical Range of Motion Velocity(mm/s)')
    plt.savefig('{}'.format(Final_DFS_LIST[i]['FileName'][1]+'Velocity'))
    r, p = stats.pearsonr(Final_DFS_LIST[i]['Speed_WAVE'], 
                          Final_DFS_LIST[i]['Speed_VIDEO'])
    
    
    ave_s_w = np.mean(abs(Final_DFS_LIST[i]['Speed_WAVE']))
    ave_s_v = np.mean(abs(Final_DFS_LIST[i]['Speed_VIDEO']))
    
    std_s_w = np.std(abs(Final_DFS_LIST[i]['Speed_WAVE']))
    std_s_v = np.std(abs(Final_DFS_LIST[i]['Speed_VIDEO']))
    
    if Final_DFS_LIST[i]['FileName'][1] == "OPEN_HOLD":
        oh_pc.append(r)
      

    elif Final_DFS_LIST[i]['FileName'][1] == "OPEN_DIS":
        od_pc.append(r)
      

    elif Final_DFS_LIST[i]['FileName'][1] == "OPEN_NORM":
        on_pc.append(r)
       
    else: 
        of_pc.append(r)
      

    

In [ ]:
# DO NOT RUN MORE THAN ONCE
parsed_df_2 = pd.DataFrame(columns=['PatientID','FileName','REP','ROM_VIDEO','ROM_WAVE', 'SpdAvg_WAVE', 
                                  'SpdAvg_VIDEO','Disp_WAVE', 'Disp_VIDEO','Time_WAVE', 'Time_VIDEO',
                                  'Speed_WAVE', 'Speed_VIDEO'])


reps = ['R1', 'R2', 'R3', 'R4', 'R5']

In [ ]:
if 0: 

In [ ]:
reps = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6']
#reps = ['R1', 'R2', 'R3']

In [ ]:
def apnd_adj (d, v, t, i):
    d[i] = d[i].append(d[i+1])
    v[i] = v[i].append(v[i+1])
    t[i] = t[i].append(t[i+1])
    d.pop(i+1)
    v.pop(i+1)
    t.pop(i+1)
    return d, v, t

# START HERE 

In [ ]:
# df at hand 
DF = Final_DFS_LIST[3]

In [ ]:
plt.plot(DF['VerDisp_WAVE'])
plt.plot(DF['VerDisp_VIDEO'])

In [ ]:
plt.plot(-DF['VerDisp_WAVE'])

In [ ]:
peaks, _ = signal.find_peaks(-DF['VerDisp_WAVE'], height= -32, distance= 50)
print(peaks)

In [ ]:
# parse the data for wave and video, displacement and velocity signals

reps_w = np.array_split(DF['VerDisp_WAVE'], peaks)
reps_w_v = np.array_split(DF['Speed_WAVE'], peaks)
reps_w_t = np.array_split(DF['Time_WAVE'], peaks)
reps_v = np.array_split(DF['VerDisp_VIDEO'], peaks)
reps_v_v = np.array_split(DF['Speed_VIDEO'], peaks)
reps_v_t = np.array_split(DF['Time_VIDEO'], peaks)
if 1:
    reps_v.pop(0)
    reps_v_v.pop(0)
    reps_v_t.pop(0)
    reps_v.pop(-1)
    reps_v_v.pop(-1)
    reps_v_t.pop(-1)

    reps_w.pop(0)
    reps_w_v.pop(0)
    reps_w_t.pop(0)
    reps_w.pop(-1)
    reps_w_v.pop(-1)
    reps_w_t.pop(-1)

print(len(reps_w))


In [ ]:
if 0: 
    reps_w.pop(3)
    reps_w_v.pop(3)
    reps_w_t.pop(3)

    reps_v.pop(3)
    reps_v_v.pop(3)
    reps_v_t.pop(3)
    print(len(reps_v), len(reps_w))


if 1: 
    reps_w.pop(-1)
    reps_w_v.pop(-1)
    reps_w_t.pop(-1)
 
    reps_v.pop(-1)
    reps_v_v.pop(-1)
    reps_v_t.pop(-1)
    print(len(reps_v), len(reps_w))
    
if 1: 
    reps_w.pop(0)
    reps_w_v.pop(0)
    reps_w_t.pop(0)
 
    reps_v.pop(0)
    reps_v_v.pop(0)
    reps_v_t.pop(0)
    print(len(reps_v),len(reps_w))

In [ ]:
plt.figure()
for i in reps_w:
    plt.plot(i)

plt.figure()
for i in reps_w_v:   
    plt.plot(i)
    
if 0:
    plt.figure()
    for i in reps_v:
        plt.plot(i)
    

plt.figure()
for i in reps_w:
    plt.plot(i)
for i in reps_v:
    plt.plot(i)
print(len(reps_v),len(reps_w))

In [ ]:
### CAUTION
#reps_w, reps_w_v, reps_w_t = apnd_adj(reps_w, reps_w_v, reps_w_t, 5)
#reps_v, reps_v_v, reps_v_t = apnd_adj(reps_v, reps_v_v, reps_v_t, 5)



In [ ]:
# ROM and AVG speed analysis 

rom_w = rom(reps_w)
rom_v = rom(reps_v)
ave_s_w = np.array([np.mean(abs(rep_v)) for rep_v in reps_w_v])
ave_s_v = np.array([np.mean(abs(rep_v)) for rep_v in reps_v_v])

print(rom_w)
print(rom_v)
print(ave_s_w)
print(ave_s_v)

In [ ]:
current_df = pd.DataFrame()

current_df['REP'] = reps
current_df['PatientID'] =  DF['PatientID'][1]
current_df['FileName'] = DF['FileName'][1]  
current_df['ROM_WAVE'] = rom_w
current_df['ROM_VIDEO'] = rom_v
current_df['SpdAvg_WAVE'] = ave_s_w
current_df['SpdAvg_VIDEO'] = ave_s_v
current_df['Time_WAVE'] = reps_w_t  
current_df['Time_VIDEO'] = reps_v_t
current_df['Disp_WAVE'] = reps_w  
current_df['Disp_VIDEO'] = reps_v  
current_df['Speed_WAVE'] = reps_w_v 
current_df['Speed_VIDEO'] = reps_v_v


In [ ]:
rep_df = current_df
for i in range(len(rep_df)):

    print(rep_df['FileName'][i] + '_' + rep_df['REP'][i] )
    plt.figure()
    
    plt.plot(rep_df['Time_WAVE'][i].values, rep_df['Disp_WAVE'][i].values, 'b', label='Ground truth (WAVE)')
    plt.plot( rep_df['Time_VIDEO'][i].values,rep_df['Disp_VIDEO'][i].values, 'r', label='Video + FAN tracking')
    
    plt.legend(loc="upper right")
    plt.xlabel('Time (s)')
    plt.ylabel('Vertical Range of Motion (mm)')
    plt.savefig('{}'.format('Open_REP_' + rep_df['REP'][i]))

In [ ]:
parsed_df_2 = parsed_df_2.append(current_df,ignore_index=True)

print(len(parsed_df_2))

In [ ]:
# MAKE SURE TO CHANGE THIS!!!!!!!!!

cvs = 'OPEN_PARSED_ANALYSIS_2.csv'
parsed_df_2.to_csv(cvs)
print(len(parsed_df_2))
print('DONE!')

In [ ]:
for i in range(len(parsed_df_2)):
    rep_df = pd.DataFrame()
    
    rep_df['Disp_VIDEO'] = parsed_df_2['Disp_VIDEO'][i]
    rep_df['Speed_VIDEO'] = parsed_df_2['Speed_VIDEO'][i]
    
    
    rep_df['PatientID'] =  parsed_df_2['PatientID'][i]
    rep_df['FileName'] = parsed_df_2['FileName'][i]  

    rep_df['ROM_VIDEO'] = parsed_df_2['ROM_VIDEO'] [i]

    rep_df['SpdAvg_VIDEO'] = parsed_df_2 ['ROM_VIDEO'][i]
    rep_df['Time_WAVE'] = parsed_df_2['Time_WAVE']
    rep_df['Time_VIDEO'] = parsed_df_2['Time_VIDEO']
    name = parsed_df_2['PatientID'][i] + '_' + parsed_df_2['FileName'][i] + '_Video_' +parsed_df_2['REP'][i] 

    cvs = 'Parsed_Data2/OPEN_'+ name +'.csv'
    rep_df.to_csv(cvs)

In [ ]:
for i in range(len(parsed_df_2)):
    rep_df = pd.DataFrame()
    
    rep_df['Disp_WAVE'] = parsed_df_2['Disp_WAVE'][i]
    rep_df['Speed_WAVE'] = parsed_df_2['Speed_WAVE'][i]
    
    
    rep_df['PatientID'] =  parsed_df_2['PatientID'][i]
    rep_df['FileName'] = parsed_df_2['FileName'][i]  

    rep_df['ROM_WAVE'] = parsed_df_2['ROM_WAVE'] [i]

    rep_df['SpdAvg_WAVE'] = parsed_df_2 ['ROM_WAVE'][i]
    name = parsed_df_2['PatientID'][i] + '_' + parsed_df_2['FileName'][i] + '_WAVE_' +parsed_df_2['REP'][i] 

    cvs = 'Parsed_Data2/OPEN_'+ name +'.csv'
    rep_df.to_csv(cvs)

In [ ]:
for i in range(len(rep_df)):

    print(rep_df['FileName'][i] + '_' + rep_df['REP'][i] )
    plt.figure()
    
    plt.plot(rep_df['Time_WAVE'][i].values, rep_df['Disp_WAVE'][i].values, 'b', label='Ground truth (WAVE)')
    plt.plot( rep_df['Time_VIDEO'][i].values,rep_df['Disp_VIDEO'][i].values, 'r', label='Video + FAN tracking')
    
    plt.legend(loc="upper right")
    plt.xlabel('Time (s)')
    plt.ylabel('Vertical Range of Motion (mm)')
    plt.savefig('{}'.format('Open_REP_' + rep_df['REP'][i]))

In [ ]:

print(len(parsed_df_2))
#parsed_df_2.head(10)

In [ ]:
deniz = parsed_df_2.loc[lambda DF: parsed_df_2['PatientID'] == 'DJ']
mk = parsed_df_2.loc[lambda DF: parsed_df_2['PatientID'] == 'MK']
rm = parsed_df_2.loc[lambda DF: parsed_df_2['PatientID'] == 'RM']


In [ ]:

#stats.pearsonr(deniz['ROM_WAVE'].values, deniz['ROM_VIDEO'].values)
    
print(stats.pearsonr(deniz['ROM_WAVE'].values, deniz['ROM_VIDEO'].values))
print(stats.spearmanr(deniz['ROM_WAVE'].values, deniz['ROM_VIDEO'].values))
print(stats.spearmanr(deniz['SpdAvg_WAVE'].values, deniz['SpdAvg_VIDEO'].values))
print()
#print(stats.pearsonr(mk['ROM_WAVE'].values, mk['ROM_VIDEO'].values))
print(stats.spearmanr(mk['ROM_WAVE'].values, mk['ROM_VIDEO'].values))
print(stats.spearmanr(mk['SpdAvg_WAVE'].values, mk['SpdAvg_VIDEO'].values))
print()
#print(stats.pearsonr(rm['ROM_WAVE'].values, rm['ROM_VIDEO'].values))
print(stats.spearmanr(rm['ROM_WAVE'].values, rm['ROM_VIDEO'].values))
print(stats.spearmanr(rm['SpdAvg_WAVE'].values, rm['SpdAvg_VIDEO'].values))
print()

print(stats.spearmanr(parsed_df_2['ROM_WAVE'].values, parsed_df_2['ROM_VIDEO'].values))
print(stats.spearmanr(parsed_df_2['SpdAvg_WAVE'].values, parsed_df_2['SpdAvg_VIDEO'].values))
print()

In [ ]:
plt.figure()
plt.scatter(deniz['ROM_WAVE'].values, deniz['ROM_VIDEO'].values, label='Participant 1')


plt.scatter(mk['ROM_WAVE'].values, mk['ROM_VIDEO'].values, label='Participant 2')

plt.scatter(rm['ROM_WAVE'].values, rm['ROM_VIDEO'].values, label='Participant 3')

plt.legend(loc="upper left")
plt.xlabel('Vertical Range of Motion, EMA-Ground truth (mm)')
plt.ylabel('Vertical Range of Motion, Video + FAN (mm)')
plt.savefig('{}'.format('OPEN_REP_CORR'))

In [ ]:
for i in range(len(deniz)):

    print(deniz['FileName'][i] + '_' + deniz['REP'][i] )
    plt.figure()
    
    plt.plot(deniz['Time_WAVE'][i].values, deniz['Disp_WAVE'][i].values, 'b', label='Ground truth (WAVE)')
    plt.plot( deniz['Time_VIDEO'][i].values,deniz['Disp_VIDEO'][i].values, 'r', label='Video + FAN tracking')
    
    plt.legend(loc="upper right")
    plt.xlabel('Time (s)')
    plt.ylabel('Vertical Range of Motion (mm)')
    plt.savefig('{}'.format('Open_REP_' + deniz['REP'][i]))

In [ ]:
plt.figure()
plt.scatter(deniz['SpdAvg_WAVE'].values, deniz['SpdAvg_VIDEO'].values, label='Participant 1')


plt.scatter(mk['SpdAvg_WAVE'].values, mk['SpdAvg_VIDEO'].values, label='Participant 2')

plt.scatter(rm['SpdAvg_WAVE'].values, rm['SpdAvg_VIDEO'].values, label='Participant 3')

plt.legend(loc="upper left")
plt.xlabel('Average Vertical Speed, EMA-Ground truth (mm/s)')
plt.ylabel('Average Vertical Speed, Video + FAN (mm/s)')
plt.savefig('{}'.format('OPEN_REP_CORR_V'))

In [ ]:
deniz.loc[deniz['FileName'] == 'OPEN_NORM', 'color'] = 1
deniz.loc[deniz['FileName'] == 'OPEN_HOLD', 'color']=2
deniz.loc[deniz['FileName'] == 'OPEN_FAST', 'color']=3
deniz.loc[deniz['FileName'] == 'OPEN_DIS', 'color']=4

In [ ]:
from matplotlib.colors import ListedColormap
colours = ListedColormap(['r','b','g','y'])
classes = ['OPEN_NORM', 'OPEN_HOLD', 'OPEN_FAST', 'OPEN_DIS']
scatter = plt.scatter(deniz['ROM_WAVE'].values, deniz['ROM_VIDEO'].values, c =deniz['color'].values, cmap=colours)
plt.legend(handles=scatter.legend_elements()[0], labels=classes)
plt.legend(loc="upper left")

plt.xlabel('Vertical Range of Motion, Ground truth (WAVE)')
plt.ylabel('Vertical Range of Motion, Video + FAN tracking')
plt.savefig('{}'.format('OPEN_REP_CORR_DENIZ'))

In [ ]:
mk.loc[mk['FileName'] == 'OPEN_NORM', 'color'] = 1
mk.loc[mk['FileName'] == 'OPEN_HOLD', 'color']=2
mk.loc[mk['FileName'] == 'OPEN_FAST', 'color']=3
mk.loc[mk['FileName'] == 'OPEN_DIS', 'color']=4

In [ ]:
from matplotlib.colors import ListedColormap
colours = ListedColormap(['r','b','g','y'])
classes = ['OPEN_NORM', 'OPEN_HOLD', 'OPEN_FAST', 'OPEN_DIS']
scatter = plt.scatter(mk['ROM_WAVE'].values, mk['ROM_VIDEO'].values, c =mk['color'].values, cmap=colours)
plt.legend(handles=scatter.legend_elements()[0], labels=classes)

In [ ]:
print(deniz['ROM_WAVE'].values)
print(deniz['ROM_VIDEO'].values)
print(deniz['FileName'].values)

In [ ]:
for df in Final_DFS_LIST:
    print(df['PatientID'][1])

In [ ]:
parsed_df.head(40)

In [ ]:
for df in Final_DFS_LIST:
    peaks, _ = signal.find_peaks(array, height= minThreshold, distance= widthapart)
    #ROM_V, V_max, V_min = rom(df['Vert_Lip_Motion'].values, peaks)
    
    
    print(df['Condition'][1])
    print(ROM_V, V_max, V_min)
    
    reps_rom_list = 
    

In [ ]:
if 0:
    video_n = sig_norm(TASK_DFS_VIDEO_100HZ_LIST[11]['Vert_Lip_Motion'])
    wave_n = sig_norm(TASK_DFS_WAVE_100HZ_LIST[9]['Vert_Lip_Motion'])

    upper_bound = len(video_n) - 10
    upper_bound2 = len(wave_n) - 10

    s = phase_align(wave_n[600:1100], video_n, [10,upper_bound])
    print('The phase shift is:{}'.format(s))

    sp = math.ceil(s) + 600


    video_shifted = sig_shift(TASK_DFS_VIDEO_100HZ_LIST[11]['Vert_Lip_Motion'],sp)

    if len(video_shifted) <= len(wave_n):
        signal_bound = upper_bound + sp
    else: 
        signal_bound = len(wave_n) - 10 

    print()
    plt.figure()

    plt.plot(TASK_DFS_WAVE_100HZ_LIST[9]['Vert_Lip_Motion'],'b')
    plt.plot(TASK_DFS_VIDEO_100HZ_LIST[11]['Vert_Lip_Motion'],'r')

    if 0: 
        plt.figure()
        a = np.gradient(TASK_DFS_WAVE_100HZ_LIST[6]['Time'])
        plt.plot(a, 'b')
        b = np.gradient(TASK_DFS_VIDEO_100HZ_LIST[4]['Time'])
        plt.plot(b, 'r')
        plt.figure()
        plt.plot(TASK_DFS_WAVE_100HZ_LIST[6]['Vert_Lip_Motion'],'b')
        plt.plot(video_shifted,'r')

        plt.figure()
        plt.scatter(TASK_DFS_WAVE_100HZ_LIST[6]['Vert_Lip_Motion'][sp:signal_bound], 
                    video_shifted[sp:signal_bound])

        

In [ ]:
reps = ['R1', 'R2', 'R3', 'R4', 'R5']


In [ ]:
parsed_df = pd.DataFrame(columns=['PatientID','FileName','REP','Disp_WAVE','Disp_VIDEO','ROM_WAVE','ROM_VIDEO','Speed_WAVE', 'Speed_VIDEO'])

In [ ]:
for df in Final_DFS_LIST:
    current_df['REP'] = reps
    current_df['PatientID'] =  df['PatientID'][1]
    current_df['FileName'] = df['FileName'][1]
    parsed_df = parsed_df.append(current_df,ignore_index=True)